# Working with Voting Records

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

In [1]:
from cdptools import Seattle

Seattle

<CDPInstance [database: <CloudFirestoreDatabase [stg-cdp-seattle]>, file_store: <GCSFileStore [stg-cdp-seattle.appspot.com]>]>

### Get all votes for a single person

Query and merge the `person` and `vote` tables.

In [2]:
import pandas as pd

# Filter down people to just Teresa
teresa = Seattle.database.select_rows_as_list("person", filters=[("full_name", "Teresa Mosqueda")])
teresa = pd.DataFrame(teresa)

# Pull voting data
votes = pd.DataFrame(Seattle.database.select_rows_as_list("vote"))

# Merge data
teresa_voting_record = teresa.merge(
    votes,
    left_on="person_id",
    right_on="person_id",
    suffixes=("_person", "_vote")
)

teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision,event_minutes_item_id,legistar_event_item_vote_id,vote_id
0,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:34:14.510429,In Favor,650b4bc0-6dfe-4c8e-b987-326e6b326d6f,46473,02ea892d-c24d-4d91-91df-a29cd6735080
1,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-21 00:29:28.257667,In Favor,9679b518-db5b-4c3e-ab54-e35a933e1ccc,46659,0b001131-b59c-413f-8d8e-07b6b5df20f6
2,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:22:20.345379,In Favor,01c4b5c4-2d02-44b2-808b-e8f71c77c70d,46725,105291a1-ce97-452c-b0ee-91ce1a314471
3,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:28:34.894883,In Favor,cfe4d044-beab-4b05-9e74-27eefdcaeda2,47003,1564337b-f145-4f61-9d14-875faa8351c6
4,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-21 00:13:15.404858,In Favor,8a5309d2-2b18-4ce2-bbb7-e66626ae9565,46978,18bdd900-23f2-42a8-b30b-5ee7fd18abff


This is great, but it's missing context of what was actually being voted on. Currently we can see that she voted on instances of `event_minutes_item`, but what were those. Follow the same procedure of `select_rows_as_list` -> `merge` to join the data of the other tables to Teresa's voting record.

In [3]:
# Get event minutes item context data
event_minutes_items = pd.DataFrame(Seattle.database.select_rows_as_list("event_minutes_item"))
minutes_items = pd.DataFrame(Seattle.database.select_rows_as_list("minutes_item"))
events = pd.DataFrame(Seattle.database.select_rows_as_list("event"))

# Merge minutes item data
event_minutes_items = event_minutes_items.merge(
    minutes_items,
    left_on="minutes_item_id",
    right_on="minutes_item_id",
    suffixes=("_event_minutes_item", "_minutes_item")
)

# Merge event data
event_minutes_items = event_minutes_items.merge(
    events,
    left_on="event_id",
    right_on="event_id",
    suffixes=("_event_minutes_item", "_event")
)

# Merge above event minutes item data with Teresa's voting record
teresa_voting_record = teresa_voting_record.merge(
    event_minutes_items,
    left_on="event_minutes_item_id",
    right_on="event_minutes_item_id",
    suffixes=("_voting_record", "_event_minutes_item")
)

# Sort by event datetime
teresa_voting_record = teresa_voting_record.sort_values(by=["event_datetime"])
teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision_voting_record,event_minutes_item_id,...,name,agenda_file_uri,body_id,created,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
0,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:34:14.510429,In Favor,650b4bc0-6dfe-4c8e-b987-326e6b326d6f,...,"Reappointment of Torgie Madison as member, Com...",http://legistar2.granicus.com/seattle/meetings...,f934cfa0-4285-404f-8072-4163df0a03b3,2019-07-20 23:33:41.354756,2019-06-24 14:00:00,4007,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
31,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:33:44.529431,In Favor,b0ba203e-7668-4241-8862-7b1a6cdb0def,...,"June 24, 2019",http://legistar2.granicus.com/seattle/meetings...,f934cfa0-4285-404f-8072-4163df0a03b3,2019-07-20 23:33:41.354756,2019-06-24 14:00:00,4007,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
30,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:33:52.379179,In Favor,2eac84c1-800a-4605-b15d-eb47542a1a3f,...,CB 119548,http://legistar2.granicus.com/seattle/meetings...,f934cfa0-4285-404f-8072-4163df0a03b3,2019-07-20 23:33:41.354756,2019-06-24 14:00:00,4007,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
25,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:33:48.434502,In Favor,5b361b64-3a51-4a47-a3dd-8eafa34e6e2e,...,"June 17, 2019",http://legistar2.granicus.com/seattle/meetings...,f934cfa0-4285-404f-8072-4163df0a03b3,2019-07-20 23:33:41.354756,2019-06-24 14:00:00,4007,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
24,2019-07-20 23:22:09.665851,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,c4ded0b8-912a-4f68-b436-0d52d5ca66e0,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 23:34:04.248787,In Favor,ad3f5be7-1dec-405f-a931-3b680d1807e0,...,Council Land Use Action to approve a temporary...,http://legistar2.granicus.com/seattle/meetings...,f934cfa0-4285-404f-8072-4163df0a03b3,2019-07-20 23:33:41.354756,2019-06-24 14:00:00,4007,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...


In [4]:
teresa_voting_record.loc[0]

created_person                                        2019-07-20 23:22:09.665851
email                                                Teresa.Mosqueda@seattle.gov
full_name                                                        Teresa Mosqueda
legistar_person_id                                                           594
person_id                                   c4ded0b8-912a-4f68-b436-0d52d5ca66e0
phone                                                               206-684-8806
website                                  http://www.seattle.gov/council/mosqueda
created_vote                                          2019-07-20 23:34:14.510429
decision_voting_record                                                  In Favor
event_minutes_item_id                       650b4bc0-6dfe-4c8e-b987-326e6b326d6f
legistar_event_item_vote_id                                                46473
vote_id                                     02ea892d-c24d-4d91-91df-a29cd6735080
created_event_minutes_item  